In [7]:
import pandas as pd
income = pd.read_csv('/home/kamlesh/AnacondaProjects/Dataquest/Data/income.csv',index_col = False )

In [8]:
income.head()

,age,workclass,fnlwgt,education,education_num,marital_status,occupation,relationship,race,sex,capital_gain,capital_loss,hours_per_week,native_country,high_income
0,39,State-gov,77516,Bachelors,13,Never-married,Adm-clerical,Not-in-family,White,Male,2174,0,40,United-States,<=50K
1,50,Self-emp-not-inc,83311,Bachelors,13,Married-civ-spouse,Exec-managerial,Husband,White,Male,0,0,13,United-States,<=50K
2,38,Private,215646,HS-grad,9,Divorced,Handlers-cleaners,Not-in-family,White,Male,0,0,40,United-States,<=50K
3,53,Private,234721,11th,7,Married-civ-spouse,Handlers-cleaners,Husband,Black,Male,0,0,40,United-States,<=50K
4,28,Private,338409,Bachelors,13,Married-civ-spouse,Prof-specialty,Wife,Black,Female,0,0,40,Cuba,<=50K


In [9]:
# Convert a single column from text categories to numbers
col = pd.Categorical(income["workclass"])
income["workclass"] = col.codes
#print(income["workclass"].head(5))
#print(income["workclass"].head(5))

for name in ["education", "marital_status", "occupation", "relationship", "race",
             "sex", "native_country", "high_income"]:
    col = pd.Categorical(income[name])
    #df.cc.astype('category').cat.codes
    income[name] = col.codes
    
income.head()    

,age,workclass,fnlwgt,education,education_num,marital_status,occupation,relationship,race,sex,capital_gain,capital_loss,hours_per_week,native_country,high_income
0,39,7,77516,9,13,4,1,1,4,1,2174,0,40,39,0
1,50,6,83311,9,13,2,4,0,4,1,0,0,13,39,0
2,38,4,215646,11,9,0,6,1,4,1,0,0,40,39,0
3,53,4,234721,1,7,2,6,0,2,1,0,0,40,39,0
4,28,4,338409,9,13,2,10,5,2,0,0,0,40,5,0


In the last mission, we learned about the basics of decision trees, including entropy and information gain. In this mission, we'll build on those concepts to construct a full decision tree in Python and use it make predictions.

We'll use the ID3 Algorithm for constructing decision trees to accomplish this. This algorithm involves recursion and an understanding of time complexity. If you're unfamiliar with these topics, we suggest trying our Data Structures and Algorithms course. We also suggest learning about lambda functions through our command line course.

In general, recursion is the process of splitting a large problem into smaller chunks. Recursive functions will call themselves, then combine the results into a final output.

Building a tree is a perfect use case for recursive algorithms. At each node, we'll call a recursive function that will split the data into two branches. Each branch will lead to a node, and the function will call itself to build the tree out.

We've created a pseudocode version of the full ID3 Algorithm below. Pseudocode is a plain-text outline of a piece of code that explains how it works. Exploring the pseudocode for an algorithm is a good way to understand it better before trying to code it.

    def id3(data, target, columns)
    1 Create a node for the tree
    2 If all values of the target attribute are 1, Return the node, with label = 1
    3 If all values of the target attribute are 0, Return the node, with label = 0
    4 Using information gain, find A, the column that splits the data best
    5 Find the median value in column A
    6 Split column A into values below or equal to the median (0), and values above the median (1)
    7 For each possible value (0 or 1), vi, of A,
    8    Add a new tree branch below Root that corresponds to rows of data where A = vi
    9    Let Examples(vi) be the subset of examples that have the value vi for A
       10    Below this new branch add the subtree id3(data[A==vi], target, columns)
       11 Return Root
       
       
We've made a minor modification to the algorithm so that it only creates two branches from each node. This will simplify the process of constructing the tree, and make it easier to demonstrate the principles it involves.

The recursive nature of the algorithm comes into play on line 10. Every node in the tree will call the id3() function, and the final tree will be the result of all of these calls       

In the last mission, we wrote functions to calculate entropy and information gain. We've loaded these functions in as calc_entropy() and calc_information_gain().

Now we need a function that returns the name of the column we should use to split a data set. The function should take the name of the data set, the target column, and a list of columns we might want to split on as input.


instructions

    Write a function named find_best_column() that returns the name of a column to split the data on. We've started to define this function for you.

    Use find_best_column() to find the best column on which to split income.
        The target is the high_income column, and the potential columns to split with are in the list columns below.
        Assign the result to income_split.


In [10]:
import numpy
import math
def calc_entropy(column):
    """
    Calculate entropy given a pandas series, list, or numpy array.
    """
    # Compute the counts of each unique value in the column
    counts = numpy.bincount(column)
    # Divide by the total column length to get a probability
    probabilities = counts / len(column)
    
    # Initialize the entropy to 0
    entropy = 0
    # Loop through the probabilities, and add each one to the total entropy
    for prob in probabilities:
        if prob > 0:
            entropy += prob * math.log(prob, 2)
    
    return -entropy
def calc_information_gain(data, split_name, target_name):
    """
    Calculate information gain given a data set, column to split on, and target
    """
    # Calculate the original entropy
    original_entropy = calc_entropy(data[target_name])
    
    # Find the median of the column we're splitting
    column = data[split_name]
    median = column.median()
    
    # Make two subsets of the data, based on the median
    left_split = data[column <= median]
    right_split = data[column > median]
    
    # Loop through the splits and calculate the subset entropies
    to_subtract = 0
    for subset in [left_split, right_split]:
        prob = (subset.shape[0] / data.shape[0]) 
        to_subtract += prob * calc_entropy(subset[target_name])
    
    # Return information gain
    return original_entropy - to_subtract


def find_best_column(data, target_name, columns):
    # Fill in the logic here to automatically find the column in columns to split on
    # data is a dataframe
    # target_name is the name of the target variable
    # columns is a list of potential columns to split on
    information_gain =[]
    for i in columns:
        gain = calc_information_gain( data,i, target_name )
        information_gain.append(gain)
    split = information_gain.index(max(information_gain))  
    return columns[split]

# A list of columns to potentially split income with
columns = ["age", "workclass", "education_num", "marital_status", "occupation", "relationship",
           "race", "sex", "hours_per_week", "native_country"]

income_split = find_best_column( income,'high_income',columns)
#income_split = columns[income_split]
print(income_split)

marital_status


Let's build up to making the full id3() function by creating a simpler algorithm that we can extend. Here's what that algorithm looks like in pseudocode:

def id3(data, target, columns)

    1 Create a node for the tree

    2 If all values of the target attribute are 1, add 1 to counter_1

    3 If all values of the target attribute are 0, add 1 to counter_0

    4 Using information gain, find A, the column that splits the data best

    5 Find the median value in column A

    6 Split A into values below or equal to the median (0), and values above the median (1)

    7 For each possible value (0 or 1), vi, of A,

    8    Add a new tree branch below Root that corresponds to rows of data where A = vi

    9    Let Examples(vi) be the subset of examples that have the value vi for A

   10    Below this new branch, add the subtree id3(data[A==vi], target, columns)

   11 Return Root

This version is very similar to the algorithm above, but lines 2 and 3 are different. Rather than storing the entire tree (which is a bit complicated), we'll just tally how many leaves end up with the label 1, and how many end up with the label 0.

We'll replicate this algorithm in code, and apply it to the same data set we just stepped through on a previous screen:


    high_income    age    marital_status

    0              20     0

    0              60     2

    0              40     1
    
    1              25     1

    1              35     2
    
    1              55     1
    
   Instructions
       Read the id3() function below and fill in the lines that say "Insert code here...".

        The function should append 1 to label_1s if the node should be a leaf, and only has 1s for high_income.
        It should append 0 to label_0s if the node should be a leaf, and only has 0s for high_income.


In [11]:
# We'll use lists to store our labels for nodes (when we find them)
# Lists can be accessed inside our recursive function, whereas integers can't.  
# Look at the python missions on scoping for more information on this topic
label_1s = []
label_0s = []

def id3(data, target, columns):
    # The pandas.unique method will return a list of all the unique values in a series
    unique_targets = pd.unique(data[target])
    
    if len(unique_targets) == 1:
        # Insert code here to append 1 to label_1s or 0 to label_0s, based on what we should label the node
        # See lines 2 and 3 in the algorithm
        if 1 in unique_targets:
            print( data )
            label_1s.append( 1 )
        else:
            print(data)
            label_0s.append ( 0)
            
        
        # Returning here is critical -- if we don't, the recursive tree will never finish, and run forever
        # See our example above for when we returned
        return 
    
    # Find the best column to split on in our data
    best_column = find_best_column(data, target, columns)
    # Find the median of the column
    #print('best column is')
    #print( best_column)
    column_median = data[best_column].median()
    
    # Create the two splits
    left_split = data[data[best_column] <= column_median]
    right_split = data[data[best_column] > column_median]
    
    # Loop through the splits and call id3 recursively
    for split in [left_split, right_split]:
        # Call id3 recursively to process each branch
        id3(split, target, columns)
    
# Create the data set that we used in the example on the last screen
data = pd.DataFrame([
    [0,20,0],
    [0,60,2],
    [0,40,1],
    [1,25,1],
    [1,35,2],
    [1,55,1]
    ])
# Assign column names to the data
data.columns = ["high_income", "age", "marital_status"]

# Call the function on our data to set the counters properly
id3(data, "high_income", ["age", "marital_status"])

print(label_1s)
print(label_0s)

   high_income  age  marital_status
0            0   20               0
   high_income  age  marital_status
3            1   25               1
   high_income  age  marital_status
4            1   35               2
   high_income  age  marital_status
2            0   40               1
   high_income  age  marital_status
5            1   55               1
   high_income  age  marital_status
1            0   60               2
[1, 1, 1]
[0, 0, 0]


# Storing the tree

Now we can store the entire tree, rather than the leaf labels only. We'll use nested dictionaries to do this. We can represent the root node with a dictionary, and branches with the keys left and right. We'll store the column we're splitting on as the key column, and the median value as the key median. Finally, we can store the label for a leaf as the key label. We'll also number each node as we go along using the number key.

We'll use the same data set we've been working with:


    `high_income   age    marital_status

    0              20     0

    0              60     2

    0              40     1

    1              25     1

    1              35     2

    1              55     1

Here's what the dictionary for the decision tree will look like:

{  
   "left":{  
      "left":{  
         "left":{  
            "number":4,
            "label":0
         },
         "column":"age",
         "median":22.5,
         "number":3,
         "right":{  
            "number":5,
            "label":1
         }
      },
      "column":"age",
      "median":25.0,
      "number":2,
      "right":{  
         "number":6,
         "label":1
      }
   },
   "column":"age",
   "median":37.5,
   "number":1,
   "right":{  
      "left":{  
         "left":{  
            "number":9,
            "label":0
         },
         "column":"age",
         "median":47.5,
         "number":8,
         "right":{  
            "number":10,
            "label":1
         }
      },
      "column":"age",
      "median":55.0,
      "number":7,
      "right":{  
         "number":11,
         "label":0
      }
   }
}

If we look at node 2 (the left branch of the root node), we see that it matches the hand exercise we completed a few screens ago. It splits, creating a right branch (node 6) with the label 1, and a left branch (node 3) that splits again.

In order to keep track of the tree, we'll need to make some modifications to id3(). The first modification involves changing the definition to pass in the tree dictionary:

def id3(data, target, columns, tree)

    1 Create a node for the tree

    2 Number the node

    3 If all of the values of the target attribute are 1, assign 1 to the label key in tree

    4 If all of the values of the target attribute are 0, assign 0 to the label key in tree

    5 Using information gain, find A, the column that splits the data best

    6 Find the median value in column A

    7 Assign the column and median keys in tree

    8 Split A into values less than or equal to the median (0), and values above the median (1)

    9 For each possible value (0 or 1), vi, of A,

       10    Add a new tree branch below Root that corresponds to rows of data where A = vi

       11    Let Examples(vi) be the subset of examples that have the value vi for A

       12    Create a new key with the name corresponding to the side of the split (0=left, 1=right).  The value of this key should be an empty dictionary.

       13    Below this new branch, add the subtree id3(data[A==vi], target, columns, tree[split_side])

       14 Return Root

Under this approach, we're now passing the tree dictionary into our id3 function and setting some keys on it. One complexity is in how we're creating the nested dictionary. For the left split, we're adding a key to the tree dictionary that looks like this:

tree["left"] = {}

For the right side, we're adding:

tree["right"] = {}

Now that we've added this key, we're able to pass our new dictionary into the recursive call to id3(). While this new dictionary will be the dictionary for that specific node, it will be tied back to the parent dictionary (because it's a key of the original dictionary).

This process will continue building up the nested dictionary. We'll be able to access the entire dictionary using the variable tree we define before the function. Think of each recursive call as building a piece of the tree, which we can then access after all of the functions have terminated.



In [12]:
# Create a dictionary to hold the tree  
# It has to be outside of the function so we can access it later
tree = {}

# This list will let us number the nodes  
# It has to be a list so we can access it inside the function
nodes = []

def id3(data, target, columns, tree):
    unique_targets = pd.unique(data[target])
    
    # Assign the number key to the node dictionary
    nodes.append(len(nodes) + 1)
    tree["number"] = nodes[-1]

    if len(unique_targets) == 1:
        # Insert code here that assigns the "label" field to the node dictionary
        if 1 in unique_targets:
            tree["label"] = 1
        else :
            tree["label"] = 0
        
        return
    best_column = find_best_column(data, target, columns)
    column_median = data[best_column].median()
    
    # Insert code here that assigns the "column" and "median" fields to the node dictionary
    tree["column"] = best_column
    tree["median"] = column_median
    
    left_split = data[data[best_column] <= column_median]
    right_split = data[data[best_column] > column_median]
    split_dict = [["left", left_split], ["right", right_split]]
    for name, split in split_dict:
        #below lines make dictionary inside dictionary 
        # tree["left"] = {}
        tree[name] = {}
        id3(split, target, columns, tree[name])

# Call the function on our data to set the counters properly
id3(data, "high_income", ["age", "marital_status"], tree)
tree

{'column': 'age',
 'left': {'column': 'age',
  'left': {'column': 'age',
   'left': {'label': 0, 'number': 4},
   'median': 22.5,
   'number': 3,
   'right': {'label': 1, 'number': 5}},
  'median': 25.0,
  'number': 2,
  'right': {'label': 1, 'number': 6}},
 'median': 37.5,
 'number': 1,
 'right': {'column': 'age',
  'left': {'column': 'age',
   'left': {'label': 0, 'number': 9},
   'median': 47.5,
   'number': 8,
   'right': {'label': 1, 'number': 10}},
  'median': 55.0,
  'number': 7,
  'right': {'label': 0, 'number': 11}}}

# Printing labels

The tree dictionary shows all of the relevant information, but it doesn't look very nice. We can fix its appearance by printing it out in a nicer format.

To do this, we'll need to recursively iterate through our tree dictionary. Any dictionary that has a label key is a leaf. Whenever we find one, we'll print out the label. Otherwise, we'll loop through the tree's left and right keys and recursively call the same function.

We also need to keep track of a depth variable. This variable will allow us to use indentation to indicate the order of the nodes. Before we print anything out, we'll prefix it with the number of spaces corresponding to the depth variable.

Here's the pseudocode:

    def print_node(tree, depth):
    1 Check for the presence of the "label" key in the tree
    2     If found, print the label and return
    3 Print out the tree's "column" and "median" keys
    4 Iterate through the tree's "left" and "right" keys
    5     Recursively call print_node(tree[key], depth+1)
    
Instructions:    
Fill in the gaps in the print_node() function that say "Insert code here...".

    Your code should iterate through both branches of the branches list (in order), and recursively call print_node().
        Don't forget to increment depth when you call print_node.

Call print_node(), and pass in tree and depth 0.    

In [13]:
def print_with_depth(string, depth):
    # Add space before a string
    prefix = "    " * depth
    # Print a string, and indent it appropriately
    print("{0}{1}".format(prefix, string))

def print_node(tree, depth):
    # Check for the presence of "label" in the tree
    if "label" in tree:
        # If found, then this is a leaf, so print it and return
        print_with_depth("Leaf: Label {0}".format(tree["label"]), depth)
        # This is critical -- without it, you'll get infinite recursion
        return
    # Print information about what the node is splitting on
    print_with_depth("{0} > {1}".format(tree["column"], tree["median"]), depth)
    
    # Create a list of tree branches
    #branches = [tree["left"], tree["right"]]
    for branch in [tree["left"], tree["right"]]:
        print_node(branch, depth+1)    
    # Insert code here to recursively call print_node on each branch
    # Don't forget to increment depth when you pass it in

print_node(tree, 0)

age > 37.5
    age > 25.0
        age > 22.5
            Leaf: Label 0
            Leaf: Label 1
        Leaf: Label 1
    age > 55.0
        age > 47.5
            Leaf: Label 0
            Leaf: Label 1
        Leaf: Label 0


We can use the scikit-learn package to fit a decision tree. The interface is very similar to other algorithms we've fit in the past.

We use the DecisionTreeClassifier class for classification problems, and DecisionTreeRegressor for regression problems. The sklearn.tree package includes both of these classes.

In this case, we're predicting a binary outcome, so we'll use a classifier.

The first step is to train the classifier on the data. We'll use the fit method on a classifier to do this.

Instructions


    Pass in income[columns] so that we only use the named columns as predictors.
    The target is the high_income column.


In [14]:
from sklearn.tree import DecisionTreeClassifier

#import data
income_df = pd.read_csv('/home/kamlesh/AnacondaProjects/Dataquest/Data/income.csv',index_col=False)
all_columns = [ "workclass", "education_num", "marital_status", "occupation", "relationship", "race", 
           "sex", "hours_per_week", "native_country"]

# Instantiate the classifier
# Set random_state to 1 to make sure the results are consistent
clf = DecisionTreeClassifier(random_state=1)

#convert categorical values
for f in all_columns:
    code = pd.Categorical(income_df[f])
    income_df[f] = code.codes
    
clf.fit(income_df[all_columns], income_df['high_income'] )    

DecisionTreeClassifier(class_weight=None, criterion='gini', max_depth=None,
            max_features=None, max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, presort=False, random_state=1,
            splitter='best')

All of the rows in income with a position up to train_max_row (but not including it) will be part of the training set.

    Make a new dataframe called train containing all of these rows.
    Make a dataframe called test containing all of the rows with a position greater than or equal to train_max_row.


In [15]:
import numpy
import math

# Set a random seed so the shuffle is the same every time
numpy.random.seed(1)

# Shuffle the rows  
# This permutes the index randomly using numpy.random.permutation
# Then, it reindexes the dataframe with the result
# The net effect is to put the rows into random order
income_df = income.reindex(numpy.random.permutation(income_df.index))

train_max_row = math.floor(income_df.shape[0] * .8)
income_df.shape
train = income_df[:train_max_row ] 
test = income_df[train_max_row :]

In [16]:
from sklearn.metrics import roc_auc_score

clf = DecisionTreeClassifier(random_state=1)
clf.fit(train[columns], train["high_income"])

predictions = clf.predict(test[columns])
roc_auc_score( test['high_income'], predictions )

0.69346563247461923

In [17]:
predictions = clf.predict(train[columns])
error = roc_auc_score( train['high_income'],predictions)
print(error)

0.947124450144


There are three main ways to combat overfitting:

    "Prune" the tree after we build it to remove unnecessary leaves.
    Use ensembling to blend the predictions of many trees.
    Restrict the depth of the tree while we're building it.

While we'll explore all of these, we'll look at the third method first.

Limiting tree depth during the building process will result in more general rules. This prevents the tree from overfitting.

We can restrict tree depth by adding a few parameters when we initialize the DecisionTreeClassifier class:

    max_depth - Globally restricts how deep the tree can go
    min_samples_split - The minimum number of rows a node should have before it can be split; if this is set to 2, for example, then nodes with 2 rows won't be split, and will become leaves instead
    min_samples_leaf - The minimum number of rows a leaf must have
    min_weight_fraction_leaf - The fraction of input rows a leaf must have
    max_leaf_nodes - The maximum number of total leaves; this will cap the count of leaf nodes as the tree is being built

Some of these parameters aren't compatible, however. For example, we can't use max_depth and max_leaf_nodes together.

Now that we know what to tweak, let's improve our model.

In [18]:
# Decision trees model from the last screen
clf = DecisionTreeClassifier(random_state=1,min_samples_split= 13)
clf.fit( train[columns], train['high_income'] )

predictions = clf.predict(test[columns])
test_auc = roc_auc_score( test['high_income'], predictions )
print(test_auc)

0.699561714515


Instructions

    Set min_samples_split to 13 when creating the DecisionTreeClassifier.
    Make predictions on the training set, compute the AUC, and assign it to train_auc.
    Make predictions on the test set, compute the AUC, and assign it to test_auc.


In [74]:
# Decision trees model from the last screen
clf = DecisionTreeClassifier(random_state=1,min_samples_split = 13)
clf.fit( train[columns], train['high_income'])

train_predictions = clf.predict(train[columns] )
train_auc = roc_auc_score( train['high_income'],train_predictions ) 
                                
test_predictions = clf.predict(test[columns])
test_auc = roc_auc_score( test['high_income'], test_predictions )

print(train_auc)
print(test_auc)

0.841898573725
0.711079143953


Instructions:

    Set max_depth to 7 and min_samples_split to 13 when creating the DecisionTreeClassifier.
    Make predictions on the training set, compute the AUC, and assign it to train_auc.
    Make predictions on the test set, compute the AUC, and assign it to test_auc.


In [19]:
# The first decision trees model we trained and tested
clf = DecisionTreeClassifier(random_state=1,max_depth = 7 ,min_samples_split = 13)
clf.fit(train[columns], train["high_income"])
predictions = clf.predict(test[columns])
test_auc = roc_auc_score(test["high_income"], predictions)

train_predictions = clf.predict(train[columns])
train_auc = roc_auc_score(train["high_income"], train_predictions)

print(test_auc)
print(train_auc)

0.743634499673
0.748037708309


Now both roc for training and test is same, lets play aggresively

    Set max_depth to 2 and min_samples_split to 100 when creating the DecisionTreeClassifier.
    Make predictions on the training set, compute the AUC, and assign it to train_auc.
    Make predictions on the test set, compute the AUC, and assign it to test_auc.


In [20]:
# The first decision tree model we trained and tested
clf = DecisionTreeClassifier(random_state=1,max_depth= 2,min_samples_split = 100)
clf.fit(train[columns], train["high_income"])
predictions = clf.predict(test[columns])
test_auc = roc_auc_score(test["high_income"], predictions)

train_predictions = clf.predict(train[columns])
train_auc = roc_auc_score(train["high_income"], train_predictions)

print(test_auc)
print(train_auc)

0.655313848188
0.662450804216


By artificially restricting the depth of our tree, we prevent it from creating a model that's complex enough to correctly categorize some of the rows. If we don't perform the artificial restrictions, however, the tree becomes too complex, fits quirks in the data that only exist in the training set, and doesn't generalize to new data.

This is known as the bias-variance tradeoff. Imagine that we take a random sample of training data and create many models. If the models' predictions for the same row are far apart from each other, we have high variance. Imagine this time that we take a random sample of the training data and create many models. If the models' predictions for the same row are close together but far from the actual value, then we have high bias.

High bias can cause underfitting -- if a model is consistently failing to predict the correct value, it may be that it's too simple to model the data faithfully.

High variance can cause overfitting. If a model varies its predictions significantly based on small changes in the input data, then it's likely fitting itself to quirks in the training data, rather than making a generalizable model.

We call this the bias-variance tradeoff because decreasing one characteristic will usually increase the other. This is a limitation of all machine learning algorithms. If you'd like to read more about the tradeoff, check out programmer Scott Fortmann-Roe's post on this topic.

Decision trees typically suffer from high variance. The entire structure of a decision tree can change if we make a minor alteration to its training data. By restricting the depth of the tree, we increase the bias and decrease the variance. If we restrict the depth too much, we increase bias to the point where it will underfit.

You'll generally need to use your intuition and manually tweak parameters to get the "right" fit.

http://scott.fortmann-roe.com/docs/BiasVariance.html


We can induce variance and see what happens with a decision tree. To introduce noise into the data, we'll add a column of random values. A model with high variance (like a decision tree) will pick up on this noise, and overfit to it. This is because models with high variance are very sensitive to small changes in input data.

In [21]:
numpy.random.seed(1)

# Generate a column containing random numbers from 0 to 4
income_df["noise"] = numpy.random.randint(4, size=income.shape[0])

# Adjust "columns" to include the noise column
columns = ["noise", "age", "workclass", "education_num", "marital_status", "occupation", 
           "relationship", "race", "sex", "hours_per_week", "native_country"]

# Make new train and test sets
train_max_row = math.floor(income.shape[0] * .8)
train = income_df.iloc[:train_max_row]
test = income_df.iloc[train_max_row:]

# Initialize the classifier
clf = DecisionTreeClassifier(random_state=1)
clf.fit(train[columns],train['high_income'])
#training is done, now predict
predictions_train = clf.predict( train[columns])
train_auc = roc_auc_score( train['high_income'], predictions_train )

# now test lo predict karo
predictions_test = clf.predict( test[columns] )
#now compare
test_auc = roc_auc_score(test['high_income'], predictions_test  )

print(train_auc)
print(test_auc)



0.975076161435
0.691406001394


As you can see above, the random noise column causes significant overfitting. Our test set accuracy decreases to .691, and our training set accuracy increases to .975.

One way to prevent overfitting is to block the tree from growing beyond a certain depth (we tried this before). Another technique is called pruning. Pruning involves building a full tree, and then removing the leaves that don't add to prediction accuracy. Pruning prevents a model from becoming overly complex. It can result in a simpler model that has higher accuracy on the testing set.

Data scientists use pruning less often than parameter optimization (what we just did) and ensembling. It's still an important technique, though, and we'll cover it in more depth down the line.


Let's go over the main advantages and disadvantages of using decision trees. The main advantages of using decision trees is that they're:

    Easy to interpret
    Relatively fast to fit and make predictions
    Able to handle multiple types of data
    Able to pick up nonlinearities in data, and usually fairly accurate

The main disadvantage of using decision trees is their tendency to overfit.

Decision trees are a good choice for tasks where it's important to be able to interpret and convey why the algorithm is doing what it's doing.

The most powerful way to reduce decision tree overfitting is to create ensembles of trees. The random forest algorithm is a popular choice for doing this. In cases where prediction accuracy is the most important consideration, random forests usually perform better.

In the next mission, we'll explore the random forest algorithm in greater depth.

### Random forest

Over the past three missions, we learned about decision trees, and looked at ways to reduce overfitting. The most powerful tool for reducing decision tree overfitting is called the random forest algorithm. In this mission, we'll learn how to construct and apply random forests.

We'll continue using the same 1994 census data set on U.S. income. It contains information on marital status, age, type of work, and more. The target column, high_income, records salaries less than or equal to 50k a year (0), and more than 50k a year (1).

You can download the data from the University of California, Irvine's website.

A random forest is a kind of ensemble model. Ensembles combine the predictions of multiple models to create a more accurate final prediction. We'll make a simple ensemble to see how they work.

Let's create two decision trees with slightly different parameters:

    One with min_samples_leaf set to 2
    One with max_depth set to 5

Then, we'll check their accuracies separately. On the next screen, we'll combine their predictions and compare the combined accuracy with the individual accuracies of both trees.


Instructions:

    Fit both clf and clf2 to the data.
        Use train[columns] as the predictors, and train["high_income"] as the target.

    Make predictions on the test set predictors (test[columns]) using both clf and clf2.

    For both sets of predictions, compute the AUC between the predictions and the actual values (test["high_income"]) using the roc_auc_score function.
    Use the print() function to display the AUC values for both.


In [95]:
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import roc_auc_score

columns = ["age", "workclass", "education_num", "marital_status", "occupation", "relationship", "race", "sex", "hours_per_week", "native_country"]

#initialized
clf_1 = DecisionTreeClassifier(random_state=1, min_samples_leaf=2)
clf_2 = DecisionTreeClassifier(random_state=1, max_depth=5)

#fitting the model
clf_1.fit(train[columns], train["high_income"])
clf_2.fit(train[columns], train['high_income'])

#predections on test data
test_predictions_1 = clf_1.predict( test[columns] )
test_predictions_2 = clf_2.predict( test[columns])

print( roc_auc_score( test['high_income'], test_predictions_1 ) )

print( roc_auc_score( test['high_income'], test_predictions_2 ) ) 

0.693844639612
0.67422521704


When we have multiple classifiers making predictions, we can treat each set of predictions as a column in a matrix. Here's an example where we have Decision Tree 1 (DT1), Decision Tree 2 (DT2), and Decision Tree 3 (DT3):

    DT1     DT2    DT3

    0       1      0

    1       1      1

    0       0      1
    
    1       0      0

Whenever we add more models to our ensemble, we just add more columns to the combined predictions. Ultimately, we don't want this matrix, though -- we want one prediction per row in the training data. To accomplish this, we'll need to create rules to convert each row of our matrix of predictions into a single number.

We want to create a Final Prediction vector that looks like this:

    DT1     DT2    DT3    Final Prediction

    0       1      0      0

    1       1      1      1

    0       0      1      0

    1       0      0      0

There are many ways to get from the output of multiple models to a final vector of predictions. One method is majority voting, in which each classifier gets a "vote," and the most commonly voted value for each row "wins." This only works if there are more than two classifiers (and ideally an odd number, so we don't have to write a rule to break ties). Majority voting is what we applied in the example above.

Because we only had two classifiers on the last screen, we'll have to use a different method to combine predictions. We'll take the mean of all of the items in a row. Right now, we're using the predict() method, which returns either 0 or 1. predict() returns something like this:

    0

    1

    0

    1

We can use the RandomForestClassifier.predict_proba() method instead, which will predict a probability from 0 to 1 that a given class is the right one for a row. Because 0 and 1 are our two classes, we'll get a matrix containing the number of rows in the income dataframe, and two columns. predict_proba() will return a result that looks like this:

    0     1

    .7    .3

    .2    .8

    .1    .9

Each row will correspond to a prediction. The first column is the probability that the prediction is a 0, and the second column is the probability that the prediction is a 1. Each row adds up to 1.

If we just take the second column, we get the average value that the classifier would predict for that row. If there's a .9 probability that the correct classification is 1, we can use the .9 as the value the classifier is predicting. This will give us a continuous output in a single vector, instead of just 0 or 1.

Then we can add together all of the vectors we get through this method, and divide the sum by the total number of vectors to get the mean prediction made across the entire ensemble for a particular row. Finally, we round off to get a 0 or 1 prediction for the row.

If we use the predict_proba() method on both classifiers from the last screen to generate probabilities, take the mean for each row, and then round the results, we'll get ensemble predictions.

In [93]:
import numpy as np
predictions = clf.predict_proba(test[columns])[:,1]
predictions2 = clf2.predict_proba(test[columns])[:,1]
final = ( predictions + predictions2 )/2
final = np.round( final )

print( roc_auc_score( test['high_income'], final ) )

0.726882697355


## Ensembling with Bagging


As we can see from the previous screen, the combined predictions of the two trees had a higher AUC than that of either tree on its own:
 
    settings 	                  test AUC
    min_samples_leaf: 2 	      0.69
    max_depth: 5 	              0.67
    combined predictions 	      0.72

To intuitively understand why this makes sense, think about two people with the same level of talent. One learned programming in college, while the other learned it on her own (let's say using Dataquest!).

If we give both of them the same project, they'll approach it in slightly different ways, due to their different experiences. They may both produce code that achieves the same result, but one may run faster in certain areas. The other may have a better interface. Even though both of them have about the same level of talent, their solutions are stronger in different areas because they approached the problem differently.

If we combine the best parts of both of their projects, we'll end up with a stronger combined project.

Ensembling is exactly the same. The models are approaching the same problem in slightly different ways, and building different trees because we used different parameters for each one. Each tree makes different predictions in different areas. Even though both trees have about the same accuracy, when we combine them, the result is stronger because it leverages the strengths of both approaches.

The more "diverse" or dissimilar the models we use to construct an ensemble are, the stronger their combined predictions will be (assuming that all of the models have about the same accuracy). Ensembling a decision tree and a logistic regression model, for example, will result in stronger predictions than ensembling two decision trees with similar parameters. That's because those two models use very different approaches to arrive at their answers.

On the other hand, if the models we ensemble are very similar in how they make predictions, ensembling will result in a negligible boost.

Ensembling models with very different accuracies generally won't improve overall accuracy. Ensembling a model with a .75 AUC and a model with a .85 AUC on a test set will usually result in an AUC somewhere in between the two original values. There's a way around this that we'll discuss later on, which we call weighting.

In [124]:
# We'll build 10 trees
tree_count = 10

# Each "bag" will have 60% of the number of original rows
bag_proportion = .6

predictions = []
for i in range(tree_count):
    # We select 60% of the rows from train, sampling with replacement
    # We set a random state to ensure we'll be able to replicate our results
    # We set it to i instead of a fixed value so we don't get the same sample in every loop
    # That would make all of our trees the same
    bag = train.sample(frac=bag_proportion, replace=True, random_state=i)
    
    # Fit a decision tree model to the "bag"
    clf = DecisionTreeClassifier(random_state=1, min_samples_leaf=2)
    clf.fit(bag[columns], bag["high_income"])
    
    # Using the model, make predictions on the test data
    x = clf.predict_proba( test[columns])[:,1] 
    
    predictions.append(clf.predict_proba(test[columns])[:,1])
    
combined = numpy.sum( predictions, axis= 0)
rounded = numpy.round(combined)

print(roc_auc_score(test["high_income"], rounded))


0.857570910145


In [123]:
test_list =[[3,5],[3,5]]
combined = numpy.sum(test_list, axis=0) / 2
print(combined)

[ 3.  5.]


Using the bagging example from the previous screen, we gained some accuracy over a single decision tree. To be exact, we achieved an AUC score of around .733 with bagging, which is an improvement over the AUC score of .688 we got without bagging:

    settings                test AUC
    min_samples_leaf: 2 	0.688
    max_depth: 2 	        0.676
    combined predictions 	0.72
    min_samples_leaf: 2, with bagging 	0.85

Let's go back to the decision tree algorithm we explored two missions ago to explain random feature subsets:

    First we pick the maximum number of features we want to evaluate each time we split the tree.
        This has to be less than the total number of columns in the data.
    Every time we split, we pick a random sample of features from the data.
    Then we compute the information gain for each feature in our random sample, and pick the one with the highest information gain to split on.

To understand how splits work, let's look at information gain or entropy. Entropy is the measure of "disorder" in the data set. If a dataset has all the same labels, they'll have low entropy. If all the labels are different, they'll have high entropy. Splits that give us more information about the data, will ideally minimize entropy. In other words, the tree will ideally split the labels into distinct groups with as little mixture possible. This'll allow the splits to give our tree more predictive power.

We're repeating the same process to select the optimal split that minimizes entropy for a node. However, we'll only evaluate a constrained set of features that we select randomly. This introduces variation into the trees, and makes for more powerful ensembles.

Below is the ID3 algorithm that we developed earlier. We'll modify it to only consider a certain subset of the features.
instructions

    Modify find_best_column to select a random sample from columns before computing information gain.
        Look where it says Insert code here.
        Each subset will have 2 items in it.
        You can use numpy.random.choice() to select a random sample.
        The first input is the list you're picking from, and the second is the number of items you want to pick.

    Be careful not to overwrite columns when you do the selection.
        The random sample should go in a different variable, and you'll have to modify some of the surrounding code to match.

    Use the print() function to display tree.


In [126]:
# Create the data set that we used two missions ago
import pandas 
data = pandas.DataFrame([
    [0,4,20,0],
    [0,4,60,2],
    [0,5,40,1],
    [1,4,25,1],
    [1,5,35,2],
    [1,5,55,1]
    ])
data.columns = ["high_income", "employment", "age", "marital_status"]

# Set a random seed to make the results reproducible
numpy.random.seed(1)

# The dictionary to store our tree
tree = {}
nodes = []

# The function to find the column to split on
def find_best_column(data, target_name, columns):
    information_gains = []
    
    # Insert your code here
    cols = numpy.random.choice(columns, 2)
    for col in columns:
        information_gain = calc_information_gain(data, col, "high_income")
        information_gains.append(information_gain)

    # Find the name of the column with the highest gain
    highest_gain_index = information_gains.index(max(information_gains))
    highest_gain = columns[highest_gain_index]
    return highest_gain

# The function to construct an ID3 decision tree
def id3(data, target, columns, tree):
    unique_targets = pandas.unique(data[target])
    nodes.append(len(nodes) + 1)
    tree["number"] = nodes[-1]

    if len(unique_targets) == 1:
        if 0 in unique_targets:
            tree["label"] = 0
        elif 1 in unique_targets:
            tree["label"] = 1
        return
    
    best_column = find_best_column(data, target, columns)
    column_median = data[best_column].median()
    
    tree["column"] = best_column
    tree["median"] = column_median
    
    left_split = data[data[best_column] <= column_median]
    right_split = data[data[best_column] > column_median]
    split_dict = [["left", left_split], ["right", right_split]]
    
    for name, split in split_dict:
        tree[name] = {}
        id3(split, target, columns, tree[name])


# Run the ID3 algorithm on our data set and print the resulting tree
id3(data, "high_income", ["employment", "age", "marital_status"], tree)
print(tree)

{'number': 1, 'column': 'employment', 'median': 4.5, 'left': {'number': 2, 'column': 'age', 'median': 25.0, 'left': {'number': 3, 'column': 'age', 'median': 22.5, 'left': {'number': 4, 'label': 0}, 'right': {'number': 5, 'label': 1}}, 'right': {'number': 6, 'label': 0}}, 'right': {'number': 7, 'column': 'age', 'median': 40.0, 'left': {'number': 8, 'column': 'age', 'median': 37.5, 'left': {'number': 9, 'label': 1}, 'right': {'number': 10, 'label': 0}}, 'right': {'number': 11, 'label': 1}}}



We can also repeat our random subset selection process in scikit-learn. We just set the splitter parameter on DecisionTreeClassifier to "random", and the max_features parameter to "auto". If we have N columns, this will pick a subset of features of size N‾‾√

, compute the Gini coefficient for each (this is similar to information gain), and split the node on the best column in the subset.

This is essentially the same thing we did on the previous screen, but with far less typing

In [127]:
# We'll build 10 trees
tree_count = 10

# Each "bag" will have 60% of the number of original rows
bag_proportion = .6

predictions = []
for i in range(tree_count):
    # We select 60% of the rows from train, sampling with replacement
    # We set a random state to ensure we'll be able to replicate our results
    # We set it to i instead of a fixed value so we don't get the same sample every time
    bag = train.sample(frac=bag_proportion, replace=True, random_state=i)
    
    # Fit a decision tree model to the "bag"
    clf = DecisionTreeClassifier(random_state=1, min_samples_leaf=2,
                                 splitter= 'random',max_features='auto')
    clf.fit(bag[columns], bag["high_income"])
    
    # Using the model, make predictions on the test data
    predictions.append(clf.predict_proba(test[columns])[:,1])

combined = numpy.sum(predictions, axis=0) / 10
rounded = numpy.round(combined)

print(roc_auc_score(test["high_income"], rounded))

0.731937714516


In [128]:
from sklearn.ensemble import RandomForestClassifier

clf = RandomForestClassifier(n_estimators=5, random_state=1, min_samples_leaf=2)

clf.fit(train[columns], train["high_income"])

predictions = clf.predict(test[columns])
print(roc_auc_score(test["high_income"], predictions))

0.735632444983



Similar to decision trees, we can tweak some of the parameters for random forests, including:

    min_samples_leaf
    min_samples_split
    max_depth
    max_leaf_nodes

These parameters apply to the individual trees in the model, and change how they are constructed. There are also parameters specific to the random forest that alter its overall construction:

    n_estimators
    bootstrap - "Bootstrap aggregation" is another name for bagging; this parameter indicates whether to turn it on (Defaults to True)

Refer to the documentation for a full list of parameters.

Tweaking parameters can increase the accuracy of the forest. The easiest tweak is to increase the number of estimators we use. This approach yields diminishing returns -- going from 10 trees to 100 will make a bigger difference than going from 100 to 500, which will make a bigger difference than going from 500 to 1000. The accuracy increase function is logarithmic, so increasing the number of trees beyond a certain number (usually 200) won't help much at all.

Increase n_estimators to 150.

In [129]:
from sklearn.ensemble import RandomForestClassifier

clf = RandomForestClassifier(n_estimators=150, random_state=1, min_samples_leaf=2)

clf.fit(train[columns], train["high_income"])

predictions = clf.predict(test[columns])
print(roc_auc_score(test["high_income"], predictions))

0.748016353725



We were able to improve the AUC from 0.735 to 0.738, but the model using 150 trees took much longer to train. While the extra training time is trivial on the data set we're working with right now, understanding this trade-off will help you when you're working with much larger data sets where the extra time could amount to hours or days!

One of the major advantages of random forests over single decision trees is that they tend to overfit less. Although each individual decision tree in a random forest varies widely, the average of their predictions is less sensitive to the input data than a single tree is. This is because while one tree can construct an incorrect and overfit model, the average of 100 or more trees will be more likely to hone in on the signal and ignore the noise. The signal will be the same across all of the trees, whereas each tree will hone in on the noise differently. This means that the average will discard the noise and keep the signal.

In the following code cell, you'll see that we've fit a single decision tree to the training set, and made predictions for both the training and testing sets. The AUC for the training set predictions is .819, while the AUC for the testing set is .714. The fact that the test AUC is much lower than the train AUC indicates that the model is overfitting.

Now let's train a similar random forest model and contrast it with what we just did.

instructions

    Fit clf to the training set and use it to make predictions on the training set.
    Then, use it to make predictions on the testing set.
    Print both AUC scores.


In [134]:
clf = DecisionTreeClassifier(random_state=1, min_samples_leaf=5)

clf.fit(train[columns], train["high_income"])

predictions = clf.predict(train[columns])
print(roc_auc_score(train["high_income"], predictions))

predictions = clf.predict(test[columns])
print(roc_auc_score(test["high_income"], predictions))

clf = RandomForestClassifier(n_estimators=150, random_state=1, min_samples_leaf=5)

clf.fit(train[columns],train['high_income'] )
predictions_train = clf.predict( train[columns]) 
print( roc_auc_score( train['high_income'], predictions_train ) )

predictions_test = clf.predict( test[columns])
print( roc_auc_score( test['high_income'], predictions_test )  )    



0.819001392492
0.714688067838
0.79249287474
0.746842822532


As we can see in the code cell from the previous screen, overfitting decreased with a random forest, and accuracy went up overall.

While the random forest algorithm is incredibly powerful, it isn't applicable to all tasks. The main strengths of a random forest are:

    Very accurate predictions - Random forests achieve near state-of-the-art performance on many machine learning tasks. Along with neural networks and gradient-boosted trees, they're typically one of the top-performing algorithms.
    Resistance to overfitting - Due to their construction, random forests are fairly resistant to overfitting. We still need to set and tweak parameters like max_depth though.

The main weaknesses of using a random forest are:

    They're difficult to interpret - Because we've averaging the results of many trees, it can be hard to figure out why a random forest is making predictions the way it is.
    They take longer to create - Making two trees takes twice as long as making one, making three takes three times as long, and so on. Fortunately, we can exploit multicore processors to parallelize tree construction. Scikit allows us to do this through the n_jobs parameter on RandomForestClassifier. We'll discuss parallelization in greater detail later on.

Given these trade-offs, it makes sense to use random forests in situations where accuracy is of the utmost importance; being able to interpret or explain the decisions the model is making isn't key. In cases where time is of the essence or interpretability is important, a single decision tree may be a better choice.